# Getting Started with PauliwordOp

In [19]:
from symmer.symplectic import PauliwordOp
import numpy as np

# 1. Basic initialization

- `PauliwordOp.from_list`
- `PauliwordOp.from_dictionary`
- `PauliwordOp.from_matrix`

In [23]:
term_list = ['ZI', 'ZZ', 'ZX', 'YZ', 'XX', 'XY']
coeffs = [1,2,3,4,5,6]

PauliwordOp.from_list(term_list, coeffs)

 1.000+0.000j ZI +
 2.000+0.000j ZZ +
 3.000+0.000j ZX +
 4.000+0.000j YZ +
 5.000+0.000j XX +
 6.000+0.000j XY

In [24]:
term_dict = {'ZI': (1+0j),
 'ZZ': (2+0j),
 'ZX': (3+0j),
 'YZ': (4+0j),
 'XX': (5+0j),
 'XY': (6+0j)}
             
PauliwordOp.from_dictionary(term_dict)

 1.000+0.000j ZI +
 2.000+0.000j ZZ +
 3.000+0.000j ZX +
 4.000+0.000j YZ +
 5.000+0.000j XX +
 6.000+0.000j XY

In [25]:
mat = np.array([[ 3.+0.j,  3.+0.j,  0.-4.j,  5.-6.j],
                   [ 3.+0.j, -1.+0.j,  5.+6.j,  0.+4.j],
                   [ 0.+4.j,  5.-6.j, -3.+0.j, -3.+0.j],
                   [ 5.+6.j,  0.-4.j, -3.+0.j,  1.+0.j]])

PauliwordOp.from_matrix(mat)

Building operator via projectors:   0%|          | 0/16 [00:00<?, ?it/s]

 1.000+0.000j ZI +
 2.000+0.000j ZZ +
 3.000+0.000j ZX +
 4.000+0.000j YZ +
 5.000+0.000j XX +
 6.000+0.000j XY

## The Symplectic Formalism

From an implementation point of view, it is beneficial to represent Pauli operators in the _symplectic_ formalism. Here, we identify an $N$-fold Pauli string $P \in \mathcal{P}_N$ with a pair of $N$-dimensional binary vectors $\vec{x}, \vec{z} \in \mathbb{Z}_2^N$, whose elements are given by
\begin{equation}
    x_n = \begin{cases} 1, & P_n \in \{X, Y\} \\ 0, & \text{otherwise} \end{cases},
    z_n = \begin{cases} 1, & P_n \in \{Z, Y\} \\ 0, & \text{otherwise} \end{cases}.
\end{equation}
Thus, defining $\vec{b} =: \vec{x} | \vec{z} \in \mathbb{Z}_2^{2N}$ (where $|$ denotes vector/matrix concatenation) together with the map
\begin{equation}\label{map_symp_to_pauli}
\begin{aligned}
    \sigma: \mathbb{Z}_2^{2N} \to{} & \mathcal{P}_N; \\
    \vec{b} \mapsto{} & i^{\vec{x} \cdot \vec{z}} \bigotimes_{n=0}^{N-1} \Big(x_n X + (1-x_n) I\Big) \Big(z_n Z + (1 - z_n) I\Big),
\end{aligned}
\end{equation}
we may reconstruct our Pauli operator $P = \sigma(\vec{b})$. In other words, the binary vectors $\vec{x}, \vec{z}$ indicate tensor factors in which there is a Pauli $X, Z$ respectively, with the additional factor $i^{\vec{x} \cdot \vec{z}}$ correcting for any incurred phases from the multiplication $XZ = -iY$.

In [46]:
print('Using the operator below:\n')
op = PauliwordOp.from_list(term_list, coeffs)
print(op); print()
print('In the symplectic picture, each term is mapped to a binary string as follows:\n')
for term in op:
    print(f'{term} -> X block: {term.X_block[0].astype(int)}, Z block: {term.Z_block[0].astype(int)}')
print('\nThe full symplectic matrix is:\n')
print(op.symp_matrix.astype(int))

Using the operator below:

 1.000+0.000j ZI +
 2.000+0.000j ZZ +
 3.000+0.000j ZX +
 4.000+0.000j YZ +
 5.000+0.000j XX +
 6.000+0.000j XY

In the symplectic picture, each term is mapped to a binary string as follows:

 1.000+0.000j ZI -> X block: [0 0], Z block: [1 0]
 2.000+0.000j ZZ -> X block: [0 0], Z block: [1 1]
 3.000+0.000j ZX -> X block: [0 1], Z block: [1 0]
 4.000+0.000j YZ -> X block: [1 0], Z block: [1 1]
 5.000+0.000j XX -> X block: [1 1], Z block: [0 0]
 6.000+0.000j XY -> X block: [1 1], Z block: [0 1]

The full symplectic matrix is:

[[0 0 1 0]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 1]]


## 1.1 Addition

In the symplectic picture, addition ammounts to a stacking of the symplectic matrices with a subsequent cleanup over potentially duplicated entries.

In [47]:
P1 = PauliwordOp.from_list(['XY', 'ZX'])
P2 = PauliwordOp.from_list(['IY', 'ZX'])

In [48]:
P_add = P1 + P2
P_add

 1.000+0.000j IY +
 2.000+0.000j ZX +
 1.000+0.000j XY

## 1.2 Multiplication

Multiplication of Pauli operators reduces to binary vector addition in the symplectic representation, however care must be taken to ensure phases are correctly accounted for; this is overlooked in much of the stabilizer code literature. Given Pauli operators $P, Q \in \mathcal{P}_N$, we may use symplectic representation to evaluate their product
\begin{equation}
\begin{aligned}
    PQ 
    ={} & \sigma(\vec{b}_P) \sigma(\vec{b}_Q) \\
    ={} & i^{\vec{x}_P \cdot \vec{z}_P + \vec{x}_Q \cdot \vec{z}_Q} \bigotimes_{n=0}^{N-1} \Bigg[
    \Big(\big(x_{P,n} + (-1)^{z_{P,n}} x_{Q,n}\big) X + \big(1-(x_{P,n} + x_{Q,n})\big) I\Big) \times
    \Big(\big(z_{P,n} + z_{Q,n}\big) Z + \big(1-(z_{P,n} + z_{Q,n})\big) I\Big) \Bigg] \\
    ={} & i^{\vec{x}_P \cdot \vec{z}_P + \vec{x}_Q \cdot \vec{z}_Q} (-1)^{\vec{z}_P \cdot \vec{x}_Q} \bigotimes_{n=0}^{N-1} \Bigg[
    \Big(\big(x_{P,n} + x_{Q,n}\big) X + \big(1-(x_{P,n} + x_{Q,n})\big) I\Big) \times
    \Big(\big(z_{P,n} + z_{Q,n}\big) Z + \big(1-(z_{P,n} + z_{Q,n})\big) I\Big) \Bigg] \\
    ={} & i^{\vec{x}_P \cdot \vec{z}_P + \vec{x}_Q \cdot \vec{z}_Q} (-i)^{(\vec{x}_P \oplus \vec{x}_Q) \cdot (\vec{z}_P \oplus \vec{z}_Q)} (-1)^{\vec{z}_P \cdot \vec{x}_Q} \sigma(\vec{b}_P \oplus \vec{b}_Q) \\
    ={} & i^{\vec{z}_P \cdot (\vec{x}_P + \vec{x}_Q) + \vec{x}_Q \cdot (\vec{z}_P + \vec{z}_Q)} (-i)^{(\vec{x}_P \oplus \vec{x}_Q) \cdot (\vec{z}_P \oplus \vec{z}_Q)} \sigma(\vec{b}_P \oplus \vec{b}_Q) \\
    %={} & (-i)^{\langle \vec{b}_P, \vec{b}_Q \rangle} \sigma(\vec{b}_P + \vec{b}_Q).
\end{aligned}
\end{equation}

In [52]:
P_mult = P1 * P2
P_mult

 1.000+0.000j II +
 0.000+1.000j ZZ +
 1.000+0.000j XI +
-1.000+0.000j YZ

## 1.3 Commutativity

Given two Pauli operators $P, Q \in \mathcal{P}_N$ with corresponding symplectic vectors $\vec{b}_P, \vec{b}_Q \in \mathbb{Z}_2^{2N}$, define the canonical \textit{symplectic form}
\begin{equation}
    \Omega =: \begin{pmatrix} 0_N & I_N \\ I_N & 0_N \end{pmatrix}
\end{equation}
and the \textit{symplectic innner product}
\begin{equation}\label{innerprod}
    \langle \vec{b}_P, \vec{b}_Q \rangle =: \vec{b}_P \Omega \vec{b}_Q^{T} = \vec{x}_P \cdot \vec{z}_Q + \vec{z}_P \cdot \vec{x}_Q.
\end{equation}
%taken modulo 2.

Pauli operators commute when they differ on an even number of tensor factors (excluding identity positions); this corresponds with mismatches between the $\vec{X}$ and $\vec{Z}$ blocks of each operator. One may count these mismatches using the inner product above, which yields a check for commutation:
\begin{equation}\label{commute_equiv}
    [P, Q] = 0 \Leftrightarrow \langle \vec{b}_P, \vec{b}_Q \rangle \equiv 0 \mod 2.
\end{equation}

Such operations are highly parallelizable in the symplectic representation. Here, for example, we may check commutation between each term of two linear combinations $L=\sum_{t=1}^{T_L} l_t P_t, M=\sum_{t=1}^{T_M} m_t Q_t$ by evaluating the inner product of their symplectic matrices 
\begin{equation}
    \langle \vec{B}_L, \vec{B}_M \rangle = \vec{X}_L \vec{Z}_M^\top + \vec{Z}_L \vec{X}_M^\top \mod 2.    
\end{equation}
The resulting matrix will be of size $T_L \times T_M$ and the ($l,m$)-th entry is zero (one) when $[P_l, Q_m] = 0$ ($\{P_l, Q_m\} = 0$).  

In [83]:
# adjacency matrix of commuting terms within a defined operator
P = PauliwordOp.from_list(['XX', 'YY', 'ZX'])
print(P.adjacency_matrix)

[[ True  True False]
 [ True  True  True]
 [False  True  True]]


In [84]:
# commutativity between operators
P1 = PauliwordOp.from_list(['XX'])
P2 = PauliwordOp.from_list(['ZZ'])
P3 = PauliwordOp.from_list(['IY'])

print(P1.commutes(P2))
print(P1.commutes(P3))

True
False


In [85]:
# termwise commutativy
P1 = PauliwordOp.from_list(['ZZ'])
P2 = PauliwordOp.from_list(['XX', 'YY', 'ZX'])


print(P1.commutes_termwise(P2))

[[ True  True False]]


## 1.4 Tensoring PauliwordOps

We can tensor PauliwordOps together...

In [95]:
P1 = PauliwordOp.from_list(['X'], [1j])
P2 = PauliwordOp.from_list(['Z'], [2])

print(P1.tensor(P2))

 0.000+2.000j XZ


... which includes linear combinations defined over different numbers of qubits:

In [96]:
P1 = PauliwordOp.from_list(['XIY', 'ZXY'], [2+1j, 3+5j])
P2 = PauliwordOp.from_list(['ZZ', 'XY'], [3+2j, 1+8j])

print(P1.tensor(P2))

-1.000+21.000j ZXYZZ +
-37.000+29.000j ZXYXY +
 4.000+7.000j XIYZZ +
-6.000+17.000j XIYXY


# 2. Exporting PauliwordOps to different representations

In [112]:
op = PauliwordOp.from_dictionary(
    {
        'XX':1,
        'ZZ':2,
        'II':3
    }
)
print(op)

 1.000+0.000j XX +
 2.000+0.000j ZZ +
 3.000+0.000j II


## 2.1 Convert to sparse matrix

In [114]:
op_sparse_matrix = op.to_sparse_matrix

print(type(op_sparse_matrix))
print(op_sparse_matrix.toarray())

<class 'scipy.sparse.csr.csr_matrix'>
[[5.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 5.+0.j]]


## 2.2 Pandas DataFrame

In [115]:
op.to_dataframe

,Pauli terms,Coefficients (real)
0,II,1.0
1,ZZ,2.0
2,XX,3.0


## 2.3 OpenFermion

In [117]:
op.to_openfermion

(3+0j) [] +
(1+0j) [X0 X1] +
(2+0j) [Z0 Z1]

## 2.3 Qiskit

In [118]:
op.to_qiskit

PauliSumOp(SparsePauliOp(['II', 'ZZ', 'XX'],
              coeffs=[3.+0.j, 2.+0.j, 1.+0.j]), coeff=1.0)